#LLM Agents with LangChain and OpenAI

In [2]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU

!pip install duckduckgo-search -qU
!pip install pyowm -qU

!pip install langchainhub -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.6 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

#Initialize OPENAI LLM

In [4]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)


#Initialize DuckDuckGo Search Tool

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()

search_tool

DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))

#Initialize OpenWeatherMap Tool

In [6]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

# Initialize OpenWeatherMap tool
weather = OpenWeatherMapAPIWrapper()

weather_tool = load_tools(["openweathermap-api"], llm)[0]

weather_tool

OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************71a69bbe, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None))

#Load Prompt Template from LangChainHub

In [7]:
from langchain import hub

# Pull the prompt template from LangChainHub
prompt = hub.pull("hwchase17/react")

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#Create Built-in Agent

In [8]:

from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search_tool, weather_tool]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)


In [9]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

#Invoke the agent executor with an example question

In [10]:
response = agent_executor.invoke({"input": "What's the weather in Kandy?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the open_weather_map tool to get the current weather in Kandy.
Action: open_weather_map
Action Input: KandyIn Kandy, the current weather is as follows:
Detailed status: scattered clouds
Wind speed: 0.67 m/s, direction: 50°
Humidity: 34%
Temperature: 
  - Current: 30.95°C
  - High: 30.95°C
  - Low: 30.95°C
  - Feels like: 30.09°C
Rain: {}
Heat index: None
Cloud cover: 33%I now know the final answer
Final Answer: The current weather in Kandy is scattered clouds with a temperature of 30.95°C.

> Finished chain.
The current weather in Kandy is scattered clouds with a temperature of 30.95°C.


In [10]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide an accurate answer.
Action: duckduckgo_search
Action Input: machine learning definitionMachine learning is a crucial subset of artificial intelligence that enables algorithms to learn from data, make predictions, ... The definition of machine learning can be defined as that machine learning gives computers the ability to learn without being explicitly programmed. Also in 1997, Tom Mitchell defined machine learning that â€œA ... Learn what machine learning is, how it works, and the different types of algorithms and applications it uses. Explore real-world examples of machine learning in action and how to build your own skills with Coursera courses. Learn what machine learning is, how it works, and why it's important in 2023. Explore the types, applications, and career paths of this AI subfield with DataCamp. Learn machine learning, a subset of AI that enables computers to learn from d

In [11]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])




> Entering new AgentExecutor chain...
I should search for the introduction to machine learning to find out how many words it typically contains.
Action: duckduckgo_search
Action Input: "introduction to machine learning word count"Introduction: Word count is a classic problem used to introduce big data processing, and in this article, ... Practical Guides to Machine Learning. 10 stories ... Bag of Words just creates a set of vectors containing the count of word occurrences in the document (reviews), while the TF-IDF model contains information on the more important words and the less important ones as well. Bag of Words vectors are easy to interpret. However, TF-IDF usually performs better in machine learning models. The word Machine Learning was first coined by Arthur Samuel in 1959. The definition of machine learning can be defined as that machine learning gives computers the ability to learn without being explicitly programmed. Also in 1997, Tom Mitchell defined machine learning tha

#Define Custom tool


In [11]:
from langchain.agents import tool

@tool
def word_count(text: str) -> int:
  """Returns the word count."""
  return len(text.split())

In [12]:
# Add the custom tool to the list of tools
tools.append(word_count)

print(tools)

[DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text')), OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************71a69bbe, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None)), StructuredTool(name='word_count', description='Returns the word count.', args_schema=<class 'langchain_core.utils.pydantic.word_count'>, func=<function word_count at 0x79ebc4103560>)]


In [13]:
# Create a new agent with the DuckDuckGo search tool, OpenWeatherMap tool  and custom word count tool
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

#Invoke the custom agent executor with example questions

In [14]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide an accurate answer.
Action: duckduckgo_search
Action Input: "definition of machine learning"Machine learning is a crucial subset of artificial intelligence that enables algorithms to learn from data, make predictions, ... The definition of machine learning can be defined as that machine learning gives computers the ability to learn without being explicitly programmed. Also in 1997, Tom Mitchell defined machine learning that â€œA ... Machine learning is a subfield of AI that uses algorithms trained on data sets to create self-learning models. Learn about the different types of machine learning, how they work, and their applications in the real world. Learn what machine learning is, how it works, and why it's important in 2023. Explore the types, applications, and career paths of this AI subfield with DataCamp. The word Machine Learning was first coined by Arthur Samuel in 1959. The de

In [15]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])




> Entering new AgentExecutor chain...
I should use the word_count function to find the number of words in the text.
Action: word_count
Action Input: "introduction to machine learning"4I now know the final answer
Final Answer: 4

> Finished chain.
4


In [16]:
response = agent_executor.invoke({"input": "what is the temperature of the city where 'lotus tower' is located?"})

print(response['output'])



> Entering new AgentExecutor chain...
I need to find the location of the Lotus Tower first before I can get the temperature.
Action: duckduckgo_search
Action Input: Lotus Tower locationAbout. The Colombo Lotus Tower serves as Sri Lanka's tallest structure which symbolizes both modernity and cultural pride. The Sinhala nickname for this monumental building is Nelum Kuluna (which translates to "Lotus Stupa"). This tower stands as the 11th tallest autonomous structure in South Asia while holding 19th position globally at 351.5 meters (1,153 feet). Location. The exact location is- Lotus Tower, AC6, DR Wijaywardane (MCCallum) Road, next to Postal Department Head Office, Colombo, Sri Lanka. Best time to visit. You can visit to the Lotus Tower any day any time. If you want to book the banquet of the meeting hall, do it during the day. The pink glasses make the hall look pink and are a ... The Colombo Lotus Tower, standing tall at 356 meters (1,168 feet), is a remarkable symbol of Sri Lanka'